In [1]:
#Generic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [3]:

import os

def get_opponent_id(game_id, team_id, df):
    opponent_data = df[(df['Game_ID'] == game_id) & (df['Team_ID'] != team_id)].iloc[0]
    return opponent_data['Team_ID']

def process_season_csv(file_path):
    df = pd.read_csv(file_path)
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    df.sort_values(by=['Team_ID', 'GAME_DATE'], inplace=True)

    #creating hte IS_HOME_TEAM feature
    df['IS_HOME_TEAM'] = df['MATCHUP'].str.contains('vs').astype(int)
    df['OPPONENT_ID'] = df.apply(lambda x: get_opponent_id(x['Game_ID'], x['Team_ID'], df), axis=1)

    # Initialize empty list to store results
    team_data_list = []

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        team_data = get_team_features(row, df)
        team_data_list.append(team_data)
        
    team_data_df = pd.DataFrame(team_data_list)

    #add back in the  OPPONENT_ID column to the team_data_df
    team_data_df['OPPONENT_ID'] = df['OPPONENT_ID']

    return team_data_df


def get_team_features(current_game, df):
    team_id = current_game['Team_ID']
    game_id = current_game['Game_ID']
    team_data = df[(df['Team_ID'] == team_id) & (df['Game_ID'] < game_id)]
    features = {'Team_ID': team_id, 'Game_ID': game_id}

    #these are the features to average
    columns_to_avg = ['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
    for col in columns_to_avg:
        features[f"AVG_{col}"] = team_data[col].mean()

    ####adding in the W, L, and W_PCT columns without modifications
    features['W'] = current_game['W']
    features['L'] = current_game['L']
    features['W_PCT'] = current_game['W_PCT']

    #compute W_PCT_PREV_10
    prev_10_games = team_data.tail(10)
    if len(prev_10_games) == 0:
        features['W_PCT_PREV_10'] = 0
    else:
        features['W_PCT_PREV_10'] = (prev_10_games['W'].max()- prev_10_games['W'].min())/ len(prev_10_games) 

    # Add IS_HOME_TEAM feature
    features['IS_HOME_TEAM'] = current_game['IS_HOME_TEAM']

    # Add HOME_TEAM_WINS label
    features['HOME_TEAM_WINS'] = int(current_game['WL'] == 'W' and current_game['IS_HOME_TEAM'])

    return features


In [8]:

def process_season_edge_csv(file_path):
    df = pd.read_csv(file_path)
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    df.sort_values(by=['Team_ID', 'GAME_DATE'], inplace=True)

    #creating the IS home feature
    df['IS_HOME_TEAM'] = df['MATCHUP'].str.contains('vs').astype(int)
    df['OPPONENT_ID'] = df.apply(lambda x: get_opponent_id(x['Game_ID'], x['Team_ID'], df), axis=1)

    #for results
    edge_data_list = []

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if row['IS_HOME_TEAM']:
            team_data = create_edge_features(row, df)
            edge_data_list.append(team_data)
        
    team_data_df = pd.concat(edge_data_list, ignore_index=True)

    #and add in the opopnent id column
    team_data_df['OPPONENT_ID'] = df['OPPONENT_ID']

    return team_data_df

def create_edge_features(current_game, df):
    team_id = current_game['Team_ID']
    game_id = current_game['Game_ID']
    opponent_id = current_game['OPPONENT_ID']
    team_data = df[(df['Team_ID'] == team_id) & (df['Game_ID'] < game_id)]
    opponent_data = df[(df['Team_ID'] == opponent_id) & (df['Game_ID'] < game_id)]
    team_features = {'Team_ID': team_id, 'Game_ID': game_id}
    opponent_features = {'Team_ID': opponent_id, 'Game_ID': game_id}

    #values to average
    columns_to_avg = ['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
    for col in columns_to_avg:
        team_features[f"AVG_{col}"] = team_data[col].mean()
        opponent_features[f"AVG_{col}"] = opponent_data[col].mean()


    ### put in the W, L, and W_PCT columns without modifications
    team_features['W'] = current_game['W']
    team_features['L'] = current_game['L']
    team_features['W_PCT'] = current_game['W_PCT']

    opp_game = df[(df['Team_ID'] == opponent_id) & (df['Game_ID'] == game_id)]
    opponent_features['W'] = opp_game['W']
    opponent_features['L'] = opp_game['L']
    opponent_features['W_PCT'] = opp_game['W_PCT']

    #trailing 10 games
    prev_10_games = team_data.tail(10)
    if len(prev_10_games) == 0:
        team_features['W_PCT_PREV_10'] = 0
    else:
        team_features['W_PCT_PREV_10'] = (prev_10_games['W'].max()- prev_10_games['W'].min())/ len(prev_10_games) 
    opp_prev_10_games = opponent_data.tail(10)
    if len(opp_prev_10_games) == 0:
        opponent_features['W_PCT_PREV_10'] = 0
    else:
        opponent_features['W_PCT_PREV_10'] = (opp_prev_10_games['W'].max()- opp_prev_10_games['W'].min())/ len(opp_prev_10_games)

    #Add IS_HOME_TEAM feature
    team_features['IS_HOME_TEAM'] = current_game['IS_HOME_TEAM']
    opponent_features['IS_HOME_TEAM'] = 0 if current_game['IS_HOME_TEAM'] else 1

    #Add HOME_TEAM_WINS label
    team_features['HOME_TEAM_WINS'] = int(current_game['WL'] == 'W' and current_game['IS_HOME_TEAM'])
    opponent_features['HOME_TEAM_WINS'] = int(current_game['WL'] == 'W' and not current_game['IS_HOME_TEAM'])

    edge_features_list = []

    #Calculate differences in features
    for col in columns_to_avg:
        edge_features_list.append(team_features[f"AVG_{col}"] - opponent_features[f"AVG_{col}"])

    ### Calculate differences in features for previous matchups
    
    prev_matchups = df[(df['Team_ID'] == team_id) & (df['OPPONENT_ID'] == opponent_id) & (df['Game_ID'] < game_id)]
    opp_prev_matchups = df[(df['Team_ID'] == opponent_id) & (df['OPPONENT_ID'] == team_id) & (df['Game_ID'] < game_id)]

    prev_matchup_features = {}
    for col in columns_to_avg:
        team_avg = prev_matchups[col].mean()
        opp_avg = opp_prev_matchups[col].mean()
        if not pd.isna(team_avg) and not pd.isna(opp_avg):
            prev_matchup_features[f"PREV_MATCHUP_DIFF_{col}"] = team_avg - opp_avg
        else:
            prev_matchup_features[f"PREV_MATCHUP_DIFF_{col}"] = 0

     # Combine edge features into a single dictionary
    edge_features = {}
    edge_features.update(team_features)
    edge_features.update(opponent_features)
    edge_features.update(prev_matchup_features)

    # Convert edge_features to a DataFrame with a single row
    edge_features_df = pd.DataFrame(edge_features, index=[0])

    return edge_features_df




In [19]:

csv_folder = "C:\\Users\\Joe\\Desktop\\Data Science Masters\\Tufts Spring 2023\\CS 150\\Final Project\\Input Data\\team"


#Process all CSV files and store results in a list
team_data_all = []

for file_name in os.listdir(csv_folder):
    if file_name.startswith("team_season_stats_") and file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder, file_name)
        season_data = process_season_csv(file_path)
        team_data_all.append(season_data)

#combine all season data into one DataFrame
team_data_df = pd.concat(team_data_all)
team_data_df.to_csv("aggr_team_data.csv", index=False)

100%|██████████| 2160/2160 [00:04<00:00, 461.45it/s]


In [9]:
csv_folder = "C:\\Users\\Joe\\Desktop\\Data Science Masters\\Tufts Spring 2023\\CS 150\\Final Project\\Input Data\\team"
#Creating edge features
edge_data_all = []


for file_name in os.listdir(csv_folder):
    if file_name.startswith("team_season_stats_") and file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder, file_name)
        season_data = process_season_edge_csv(file_path)
        edge_data_all.append(season_data)

edge_features_df = pd.concat(edge_data_all)
edge_features_df.to_csv("edge_features.csv", index=False)

100%|██████████| 2160/2160 [00:12<00:00, 167.65it/s]


In [11]:
len(edge_features_df)

8460

In [33]:
test_list = set(team_data_df['Game_ID'].to_list())
print(len(test_list))

8460


In [35]:
8460/(82*.5*30)

6.878048780487805

In [ ]:
edge_features_df.head(8)

In [ ]:


############ PLAYER DATA ############


In [1]:
def process_player_data_csv(file_path):
    df = pd.read_csv(file_path)
    df.sort_values(by=['PLAYER_ID', 'GAME_ID'], inplace=True)
    df['MIN_x'] = df['MIN_x'].apply(lambda x: int(x.split('.')[0].split(':')[0]) + int(x.split(':')[1])/60 if pd.notnull(x) and isinstance(x, str) and ':' in x else x)



    #Initialize empty list to store results
    player_data_list = []

    for index, row in tqdm(df.iterrows(), desc="Processing rows", total=df.shape[0]):
        player_data = get_player_features(row, df)
        player_data_list.append(player_data)

    return pd.DataFrame(player_data_list)

def get_player_features(current_game, df):
    player_id = current_game['PLAYER_ID']
    game_id = current_game['GAME_ID']
    team_id = current_game['TEAM_ID']
    player_data = df[(df['PLAYER_ID'] == player_id) & (df['GAME_ID'] < game_id)]
    features = {'PLAYER_ID': player_id, 'Game_ID': game_id, 'Team_ID' : team_id}

    #Compute averages for relevant columns
    columns_to_avg = ['MIN_x', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE']
    for col in columns_to_avg:
        features[f"AVG_{col}"] = player_data[col].mean()

    #Add START_POSITION and DID_NOT_PLAY features
    features['START_POSITION'] = current_game['START_POSITION_x']
    features['DID_NOT_PLAY'] = 1 if pd.notnull(current_game['COMMENT_x']) else 0

    return features

In [3]:
import pandas as pd
import os as os
from tqdm import tqdm

csv_folder = "C:\\Users\\Joe\\Desktop\\Data Science Masters\\Tufts Spring 2023\\CS 150\\Final Project\\Input Data\\player"


player_data_all = []

#Process player data CSV file and store results in a list using our function
for file_name in os.listdir(csv_folder):
    if file_name.startswith("player_game_stats_") and file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder, file_name)
        player_data_df = process_player_data_csv(file_path)
        player_data_all.append(player_data_df)

player_data_df_all = pd.concat(player_data_all)

player_data_df_all.to_csv("aggr_player_data.csv", index=False)



Processing rows: 100%|██████████| 33356/33356 [02:25<00:00, 229.87it/s]


In [4]:
player_data_df_all.head()

,PLAYER_ID,Game_ID,Team_ID,AVG_MIN_x,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,...,AVG_TS_PCT,AVG_USG_PCT,AVG_E_USG_PCT,AVG_E_PACE,AVG_PACE,AVG_PACE_PER40,AVG_POSS,AVG_PIE,START_POSITION,DID_NOT_PLAY
0,708,21300004,1610612751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,0
1,708,21300028,1610612751,26.283333,4.000000,8.00,0.500000,0.0,0.000000,0.0,...,0.5000,0.179000,0.17700,97.320000,94.980000,79.150000,52.000000,0.118,F,0
2,708,21300043,1610612751,26.041667,3.500000,9.50,0.386500,0.0,0.000000,0.0,...,0.3865,0.210000,0.21300,97.815000,98.655000,82.210000,54.000000,0.068,F,0
3,708,21300052,1610612751,24.833333,3.333333,10.00,0.348667,0.0,0.333333,0.0,...,0.3700,0.232333,0.23500,98.743333,99.316667,82.763333,51.333333,0.059,F,0
4,708,21300077,1610612751,22.025000,3.000000,8.75,0.361500,0.0,0.250000,0.0,...,0.3775,0.232000,0.23525,96.435000,96.987500,80.822500,44.750000,0.076,F,0


In [5]:
unique_player_ids_player_data = set(player_data_df_all['PLAYER_ID'].unique())


In [6]:
from nba_api.stats.endpoints import commonplayerinfo
import time



player_ids = unique_player_ids_player_data

#Empty DataFrame to store player information
player_info_df = pd.DataFrame(columns=['PLAYER_ID', 'HEIGHT', 'WEIGHT', 'BIRTHDATE'])

#Fetch player information
for player_id in tqdm(player_ids, desc="Fetching player info", total=len(player_ids)):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    player_info_data = player_info.get_data_frames()[0].iloc[0]
    time.sleep(0.6) ## need this for the API

    player_data = {
        'PLAYER_ID': player_id,
        'HEIGHT': player_info_data['HEIGHT'],
        'WEIGHT': player_info_data['WEIGHT'],
        'BIRTHDATE': player_info_data['BIRTHDATE']
    }
    
    player_info_df = player_info_df.append(player_data, ignore_index=True)

#Print the DataFrame
print(len(player_info_df))
print(len(unique_player_ids_player_data))


Fetching player info:   0%|          | 0/1710 [00:00<?, ?it/s]C:\Users\Joe\AppData\Local\Temp\ipykernel_9260\3197184227.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_info_df = player_info_df.append(player_data, ignore_index=True)
Fetching player info:   0%|          | 1/1710 [00:00<27:08,  1.05it/s]C:\Users\Joe\AppData\Local\Temp\ipykernel_9260\3197184227.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_info_df = player_info_df.append(player_data, ignore_index=True)
Fetching player info:   0%|          | 2/1710 [00:01<24:57,  1.14it/s]C:\Users\Joe\AppData\Local\Temp\ipykernel_9260\3197184227.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_info_df = player_info_df.append(player_data, igno

1710
1710


In [7]:
len(player_info_df[player_info_df['HEIGHT'].isnull() | player_info_df['HEIGHT'].eq("")])
#509

509

In [8]:

#this data is incomplete so can't really use it....
import datetime as dt

def height_to_inches(height):
    try:
        feet, inches = height.split('-')
        return int(feet) * 12 + int(inches)
    except:
        pass
    

def birthdate_to_days(birthdate):
    return (pd.to_datetime(birthdate) - pd.Timestamp("1970-01-01")) // pd.Timedelta("1D")

#Convert height to inches
player_info_df['HEIGHT'] = player_info_df['HEIGHT'].apply(height_to_inches)

#Convert birthdate to an integer (days since 1970-01-01)
player_info_df['BIRTHDATE'] = player_info_df['BIRTHDATE'].apply(birthdate_to_days)

print(player_info_df.head())


  PLAYER_ID  HEIGHT WEIGHT  BIRTHDATE
0   1630208    84.0    245      10194
1   1630210    70.0    175      10653
2   1630211    75.0    190      11093
3   1630213     NaN             10239
4   1630214    79.0    245      10603


In [9]:
player_bday_df = player_info_df[['PLAYER_ID', 'BIRTHDATE']]

In [10]:
player_bday_df['BIRTHDATE'].eq('').sum()
#0

0

In [11]:
player_data_all_df_with_bday = player_data_df_all.merge(player_bday_df, on='PLAYER_ID', how='left')

In [12]:
len(player_data_all_df_with_bday)
#250831

250831

In [13]:
player_data_all_df_with_bday.head()

,PLAYER_ID,Game_ID,Team_ID,AVG_MIN_x,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,...,AVG_USG_PCT,AVG_E_USG_PCT,AVG_E_PACE,AVG_PACE,AVG_PACE_PER40,AVG_POSS,AVG_PIE,START_POSITION,DID_NOT_PLAY,BIRTHDATE
0,708,21300004,1610612751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,0,2330
1,708,21300028,1610612751,26.283333,4.000000,8.00,0.500000,0.0,0.000000,0.0,...,0.179000,0.17700,97.320000,94.980000,79.150000,52.000000,0.118,F,0,2330
2,708,21300043,1610612751,26.041667,3.500000,9.50,0.386500,0.0,0.000000,0.0,...,0.210000,0.21300,97.815000,98.655000,82.210000,54.000000,0.068,F,0,2330
3,708,21300052,1610612751,24.833333,3.333333,10.00,0.348667,0.0,0.333333,0.0,...,0.232333,0.23500,98.743333,99.316667,82.763333,51.333333,0.059,F,0,2330
4,708,21300077,1610612751,22.025000,3.000000,8.75,0.361500,0.0,0.250000,0.0,...,0.232000,0.23525,96.435000,96.987500,80.822500,44.750000,0.076,F,0,2330


In [14]:
player_data_all_df_with_bday.to_csv("aggr_player_data_full.csv", index=False)